# PyVO Image Access
Alex Drlica-Wagner

This is a rough sketch of a notebook demonstrating how images can be accessed using PyVO tools. Some useful references listed below.

1. [lsst.rsp.catalog](https://github.com/lsst-sqre/lsst-rsp/blob/main/src/lsst/rsp/catalog.py) showing how the TAP authentication is done on the RSP.
2. [Community post on external table access](https://community.lsst.org/t/will-there-be-external-tap-access-to-rsp-dp0-2-tables/6660) with more useful information about authentication.
3. [PyVO Data Access](https://pyvo.readthedocs.io/en/latest/dal/index.html)
4. [The RSP Portal Aspect](https://data-int.lsst.cloud/portal/app/?__action=layout.showDropDown&view=TAPSearch)

In [ ]:
import matplotlib.pyplot as plt
import gc
import numpy as np
import requests

# Astropy imports
from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize,  ZScaleInterval
from astropy.visualization.stretch import SinhStretch, LinearStretch, SqrtStretch

# PyVO
import pyvo
from pyvo.dal.adhoc import DatalinkResults

# Image visualization routines.
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
from lsst.afw.image.exposure import ExposureF

# Import the Rubin TAP service utilities
import lsst.rsp
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_access_token

plt.style.use('tableau-colorblind10')
%matplotlib inline
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 14,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [8, 8],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

## Conventional Butler Image Access

Here we'll grab a `deepCoadd_calexp` in the conventional manner using the Butler.

In [ ]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collection)

# Note: This will trigger a warning from CFITSIO in w_2022_22.
# This warning can be safely ignored and will be corrected in the future.

In [ ]:
dataId = {'tract': 3828, 'patch': 21, 'band': 'i'}
datasetType='deepCoadd_calexp'
coadd = butler.get(datasetType,**dataId)

In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(coadd.maskedImage)
plt.show()

## PyVO Image Access

Now we'll grab the same image using tools from PyVO. This mimics what the Portal is doing with its "Image Search (OpsTAP)".

The basic workflow is:
1. Use the TAP service to download image metadata associated with images that satisfy a subset of criteria. In particular, we are after the `access_url`.
2. The `access_url` is actually a datalink to a dynamically constructed VOTable that itself contains a link to the actual URL. This second url is signed by Google, so it can be directly accessed for a limited time. We create a DatalinkResults object to be able to access this second URL.
3. We download the image and play around with it a bit.


In [ ]:
# First, get our RSP access token (we will need this to download the data)
token = get_access_token()

# Get an instance of the TAP service
service = get_tap_service()

In [ ]:
# Query for images satisfying some criteria
query = """
SELECT * 
FROM ivoa.ObsCore 
WHERE calib_level = 3 AND dataproduct_type = 'image' AND dataproduct_subtype = 'lsst.deepCoadd_calexp' 
-- Selects the coadd containing this point
AND CONTAINS(POINT('ICRS', 57.5, -36.4), s_region)=1
-- Selects the i-band image (691 nm < lambda < 818 nm)
AND ( 700 BETWEEN em_min AND em_max )
"""

# Execute the query and get the results
result_set = service.search(query)
result_table = result_set.to_table()
result_table

In [ ]:
# Let's take a look at the result in the first row
row = result_set[0]
print(row)
print("Datalink URL:",row.getdataurl())

In [ ]:
# Next, create a session with datalink authentication. 
# This draws on the pattern that is used for TAP access by lsst.rsp.
# https://github.com/lsst-sqre/lsst-rsp/blob/main/src/lsst/rsp/catalog.py

# Create a datalink authentication
datalink_url = 'https://data-int.lsst.cloud/api/datalink'
s = requests.Session()
s.headers["Authorization"] = "Bearer " + token
auth = pyvo.auth.authsession.AuthSession()
auth.credentials.set("lsst-token", s)
auth.add_security_method_for_url(datalink_url, "lsst-token")
auth.add_security_method_for_url(datalink_url + "/links", "lsst-token")

In [ ]:
# Now we can create the datalink, which will allow us to access the (transient) signed URL for image access
datalink = DatalinkResults.from_result_url(row.getdataurl(),session=auth)
print(datalink.status)
print(datalink.votable)

# Now we can grab the google signed URL for the image (note that this will expire)
image_url = datalink.getrecord(0).get('access_url')
print(image_url)

In [ ]:
# Now we can grab the image file from the URL
filename = download_file(image_url)
hdulist = fits.open(image_url)

# We could also open the image directly from the URL
#hdulist = fits.open(image_url)

# Looks like a deepCoadd_calexp to me...
for hdu in hdulist:
    print(hdu.name)

In [ ]:
# Let's plot the image and see what it looks like...
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', interval=ZScaleInterval(), stretch=SqrtStretch(), cmap='gray')
fig.colorbar(im[0])

In [ ]:
# We can also create an ExposureF stack object
new_coadd = ExposureF(filename)

fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(new_coadd.maskedImage)
plt.show()